In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import zipfile

In [2]:
# Extract images from the two zip files

!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

# Function to remove spaces from file names in a given directory
def remove_spaces_in_filenames(directory):
    for filename in os.listdir(directory):
        if ' ' in filename:
            new_filename = filename.replace(' ', '_')
            os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
            print(f'Renamed: {filename} -> {new_filename}')

# Function to extract images from a zip file and return a list of image file paths
def extract_images(zip_file, extract_to):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        return [os.path.join(extract_to, file) for file in zip_ref.namelist() if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Function to combine images and save them to a new zip file
def combine_images_to_zip(image_files, output_zip):
    with zipfile.ZipFile(output_zip, 'w') as zipf:
        for image_file in image_files:
            zipf.write(image_file, os.path.basename(image_file))

# Specify the directory containing the files
directory = '/content/Qualcomm-DL-Hackathon/train'

# Remove spaces from file names
remove_spaces_in_filenames(directory)

zip_file1 = '/content/Qualcomm-DL-Hackathon/train/images_part-1.zip'
zip_file2 = '/content/Qualcomm-DL-Hackathon/train/images_part-2.zip'

extract_to1 = 'extracted_zip1'
extract_to2 = 'extracted_zip2'

image_files1 = extract_images(zip_file1, extract_to1)
image_files2 = extract_images(zip_file2, extract_to2)

# Combine the images and save them to a new zip file
combined_image_files = image_files1 + image_files2
output_zip = 'images.zip'
combine_images_to_zip(combined_image_files, output_zip)

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 17.80 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Renamed: images part-1.zip -> images_part-1.zip
Renamed: images part-2.zip -> images_part-2.zip


In [3]:
extract_dir = "/content/images"
os.makedirs(extract_dir, exist_ok=True)

# Open and extract the zip file
with zipfile.ZipFile("images.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Contents extracted to {extract_dir}")

Contents extracted to /content/images


In [4]:
# Emergency Dataset Class
class EmergencyDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        label = int(self.data_frame.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

In [5]:
# Split train.csv into train and test sets
train_csv = '/content/Qualcomm-DL-Hackathon/train/train.csv'
data = pd.read_csv(train_csv)
# Split the data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=50)

# Further split the train_data into train and validation sets
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=50)

# Save the split data into new CSV files
train_data.to_csv('train_split.csv', index=False)
val_data.to_csv('val_split.csv', index=False)
test_data.to_csv('test_split.csv', index=False)

print("Data has been split into train_split.csv, val_split.csv, and test_split.csv")

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Data has been split into train_split.csv, val_split.csv, and test_split.csv


In [6]:
# Load Data
root_dir = '/content/images'
train_dataset = EmergencyDataset(csv_file='train_split.csv', root_dir=root_dir, transform=transform)
val_dataset = EmergencyDataset(csv_file='val_split.csv', root_dir=root_dir, transform=transform)
test_dataset = EmergencyDataset(csv_file='test_split.csv', root_dir=root_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Initially freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last layer
for param in model.layer4.parameters():
    param.requires_grad = True

# Modify the fully connected layer
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 1),  # Output layer with 1 unit for binary classification
    nn.Sigmoid()  # Sigmoid activation for binary classification
)
model = model.to(device)

# Define different learning rates for different layers
optimizer = optim.Adam([
    {'params': model.layer4.parameters(), 'lr': 1e-4},
    {'params': model.fc.parameters(), 'lr': 1e-3}
])

# Define the loss function
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 145MB/s]


In [8]:
num_epochs = 10
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Reshape labels to match the output shape
        labels = labels.view(-1, 1).float()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    # Validation step
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_labels = val_labels.view(-1, 1).float()
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_labels).item()
            val_predicted = (val_outputs > 0.5).float()
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    val_loss /= len(val_loader)
    val_acc = 100 * val_correct / val_total
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

    # Save the model if validation loss has decreased
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'vehicle_classifier.pth')
        print("Model saved!")

# Load the best model
model.load_state_dict(torch.load('vehicle_classifier.pth'))

Epoch 1/50, Loss: 0.3287, Accuracy: 84.89%
Validation Loss: 0.1530, Validation Accuracy: 94.32%
Model saved!
Epoch 2/50, Loss: 0.0930, Accuracy: 97.43%
Validation Loss: 0.1372, Validation Accuracy: 94.70%
Model saved!
Epoch 3/50, Loss: 0.0322, Accuracy: 99.43%
Validation Loss: 0.1337, Validation Accuracy: 96.21%
Model saved!
Epoch 4/50, Loss: 0.0173, Accuracy: 99.81%
Validation Loss: 0.1571, Validation Accuracy: 93.94%
Epoch 5/50, Loss: 0.0083, Accuracy: 99.81%
Validation Loss: 0.1575, Validation Accuracy: 94.32%
Epoch 6/50, Loss: 0.0117, Accuracy: 99.81%
Validation Loss: 0.1363, Validation Accuracy: 95.08%
Epoch 7/50, Loss: 0.0076, Accuracy: 99.90%
Validation Loss: 0.1525, Validation Accuracy: 95.83%
Epoch 8/50, Loss: 0.0046, Accuracy: 99.90%
Validation Loss: 0.1487, Validation Accuracy: 95.83%
Epoch 9/50, Loss: 0.0029, Accuracy: 99.90%
Validation Loss: 0.1532, Validation Accuracy: 95.08%
Epoch 10/50, Loss: 0.0029, Accuracy: 99.90%
Validation Loss: 0.1565, Validation Accuracy: 94.32%


<ipython-input-8-68fc0f57b2ca>:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('vehicle_classifier.pth'))


<All keys matched successfully>

In [9]:
# Custom Dataset Class for Test Data
class TestDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, self.data_frame.iloc[idx, 0]

# Load the test data
test_csv = '/content/Qualcomm-DL-Hackathon/test.csv'
test_dataset = TestDataset(csv_file=test_csv, root_dir=root_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Perform inference on the test data and write results to sample_submissions.csv
results = []

with torch.no_grad():
    for inputs, img_names in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        for img_name, pred in zip(img_names, predicted):
            results.append([img_name, pred.item()])

# Save the results to sample_submissions.csv
submission_df = pd.DataFrame(results, columns=['image_names', 'emergency_or_not'])
submission_df.to_csv('sample_submissions.csv', index=False)

print("Results have been written to sample_submissions.csv")

Results have been written to sample_submissions.csv
